# Training a model
see README.md for details

Loosely following this tutorial:
https://medium.com/nlplanet/a-full-guide-to-finetuning-t5-for-text2text-and-building-a-demo-with-streamlit-c72009631887

In [4]:
import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
from tqdm import tqdm
from glob import glob

import torch
import wandb
from datasets import Dataset
from evaluate import load
from transformers import AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer

from config import data_root, model_root, checkpoint_name
from config import token_len, annot_len

2024-07-15 15:55:25.464006: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-15 15:55:25.464067: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-15 15:55:25.465132: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-15 15:55:25.471738: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 15:55:26.189136: W tensorflow/compiler/tf2

In [5]:
torch.cuda.is_available()

True

In [6]:
# num_special_tokens = 3
model_checkpoint = "google/byt5-small"

In [7]:
dfs = []
for fname in glob(f"{data_root}/*.csv"):
    dfs += [pd.read_csv(fname, names=["train","test"])]
df = pd.concat(dfs, axis=0)
ds = Dataset.from_pandas(df)
# ds = load_dataset("csv", data_files=glob(f"{data_root}/*.csv"), features=["input","output"])

# small_train_dataset = ds["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = ds["test"].shuffle(seed=42).select(range(300))

ds.features

{'train': Value(dtype='string', id=None),
 'test': Value(dtype='string', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [8]:
# https://wandb.ai/mapto-university-of-milan/ocs-lemmatizer

# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="ocs-lemmatizer"

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="true"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"

wandb.init(project='ocs-lemmatizer')


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f469c5f7d90>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f469c590050, execution_count=8 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f469c592fd0, raw_cell="# https://wandb.ai/mapto-university-of-milan/ocs-l.." store_history=True silent=False shell_futures=True cell_id=f25c007d-4798-445b-95ad-183ddfbc0c3c> result=<wandb.sdk.wandb_run.Run object at 0x7f469c6011d0>>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [8]:
exact_match_metric = load("exact_match")

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_root}/byT5-ocs-ue",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    # torch_empty_cache_steps=100,
    disable_tqdm=False,
    report_to=None,  # disable wandb.ai
)


model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [9]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    # train_dataset=small_train_dataset,
    # eval_dataset=small_eval_dataset,    
    #     data_collator=data_collator,
    compute_metrics=exact_match_metric,
)

In [10]:
trainer.train()
# wandb.finish()

TypeError: vars() argument must have __dict__ attribute